# 6. Kaavatietomallin implementointi

## 6.1. Geopackagesta JSON-tiedostoiksi

ETL-työkalussa [kaavatietomallin](https://tietomallit.ymparisto.fi/kaavatiedot/v1.1/looginenmalli/uml/doc/) implementointi tapahtuu muuntamalla GeoPandas GeoDataFrame -muotoinen kaava-aineisto JSON-formaattiin kunnittain. ETL-muuntimelle ilmoitetaan aineistolähteeksi joko “kunta” tai “KTJ” sekä molempien aineistolähteiden kaavatunnukset. Osana tietojen vertailua ETL-muuntimen osassa 4.3 on luotu tietokenttä “refe_kaavatunnus”, joka kertoo esim. KTJ-pohjaisessa aineistossa kaavan vastaavan kuntatunnuksen. Mikäli vastaavaa tunnusta ei löydy (esim. kaavakohde on vain KTJ:n aineistossa), ETL-muunnin jättää kunnan indeksitunnus-tiedon JSON:ssa tyhjäksi.

In [ ]:
# Käytettävien Python moduulien ja kehitettyjen funktioiden sisäänluku
import geopandas as gpd
import json
from lib.implement_kaavatietomalli import dataToJSON

In [ ]:
# Kaava-aineisto
data = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")
kunta = data.loc[data['kuntakoodi'] == '<insert kuntakoodi here>']

"""
# Tee tässä tarvittaessa aineiston täydentäminen esim. kunnan kaavaindekseillä, joita ei ole KTJ-aineistossa mukana

additional_data = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")
additional_kunta = additional_data.loc[additional_data['kuntakoodi'] == '<insert kuntakoodi here>']

add_list = ['<insert tunnus here>', '<insert tunnus here>'] # listaa ne kaavatunnukset, jotka haluat loppuaineistoon mukaan!
add_ons = additional_kunta.loc[additional_kunta['<insert kaavatunnus column here>'].isin(add_list)] #vaihda kenttänimi tarvittaessa!
kunta = kunta.append(add_ons, ignore_index = True)
"""

# Jos haluat tuottaa loppuaineiston kaavatasolla kaavaindeksitason sijasta
# HUOM! Tämä sotkee kohderekisteriyksiköt, joten hae ne tarvittaessa dissolven jälkeen uudestaan (katso 2.3.1 ja 2.3.2)
kunta_dissolve = kunta.dissolve(by='<insert kaavatunnus column here>', as_index=False) 

# Aineiston muuntaminen kaavatietomalliin ja JSON-dumpiksi
data_json = dataToJSON(kaavadata=kunta,
                          aineistolahde='<insert aineistolahde here>', # joko "kunta" tai "KTJ"
                          ktj_kaavatunnus='kaavatunnus_1', # vaihda, jos olet tenhyt vertailun kuntapohjaisesti!
                          kunta_kaavatunnus='refe_kaavatunnus') # vaihda, jos olet tenhyt vertailun kuntapohjaisesti!

# JSON tallennus
with open(r'<insert  filepath here>.json', 'w', encoding='utf8') as outfile:
    result = json.dumps(data_json, indent=4, ensure_ascii=False)
    outfile.write(result)

## 6.2. Extra: JSON-tiedostojen normalisointi geopackageiksi

<span style="color:red">**Tämä vaihe ei ole pakollinen!**</span>

Kaavatietomallin mukainen JSON-aineisto voi olla kunnille hankalaa luettavaa sellaisenaan. Tarvittaessa JSON-muotoisen aineiston voi normalisoida kaavatietomallimuodossa takaisin geopackageksi.

In [ ]:
# Kehitettyjen funktioiden sisäänluku
from lib.implement_kaavatietomalli import jsonToGPKG

In [ ]:
# Normalisoi JSON geopackageksi
normalized_data = jsonToGPKG(inputfp=r"<insert filepath here>.json",
                             outputfp=r"<insert filepath here>.gpkg")